<a href="https://colab.research.google.com/github/DavyCielen/BAI_knowledge_base/blob/main/SuperRes_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Neural Image Super-Resolution <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">ESRGAN</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralImageSuperResolution" target="_blank"><font color="#999" size="4">Github</font></a>

Colab for [JoeyBallentine's fork](https://github.com/JoeyBallentine/ESRGAN) of [BlueAmulet's fork](https://github.com/BlueAmulet/ESRGAN) of [ESRGAN](https://github.com/xinntao/ESRGAN), an implementation of [Enhanced Super-Resolution Generative Adversarial Networks](https://arxiv.org/abs/1809.00219) by Xintao Wang et al. This notebook includes a few additional features.

- Enter all file/directory paths relative to your Google Drive root.
- `image_or_directory_to_upscale` may be a file path or a directory path. If a directory path is given, all images in the given directory will be processed.
- Leave `output_dir` blank to save the processed superres image(s) in the same (`image_or_directory_to_upscale`) directory.

In [ ]:
!apt install -y gmic

In [27]:
!mkdir -p /content/Colab-ESRGAN/models/
!wget --no-check-certificate "https://huggingface.co/databuzzword/esrgan/resolve/main/RRDB_ESRGAN_x4.pth" -O /content/___Colab-ESRGAN/models/1x_DeJpeg_Fatality_PlusULTRA.pth

--2025-04-03 06:48:48--  https://huggingface.co/databuzzword/esrgan/resolve/main/RRDB_ESRGAN_x4.pth
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/databuzzword/esrgan/65fece06e1ccb48853242aa972bdf00ad07a7dd8938d2dcbdf4221b59f6372ce?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27RRDB_ESRGAN_x4.pth%3B+filename%3D%22RRDB_ESRGAN_x4.pth%22%3B&Expires=1743666528&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzY2NjUyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kYXRhYnV6endvcmQvZXNyZ2FuLzY1ZmVjZTA2ZTFjY2I0ODg1MzI0MmFhOTcyYmRmMDBhZDA3YTdkZDg5MzhkMmRjYmRmNDIyMWI1OWY2MzcyY2U%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=tDjNaCaQBTTIdkGdGIMvDLJHKfiw7eRLP5UNZNBR1c4WRypgQ-jSzz3c%7EB5svCl%7Ey07hXk15KbMSXV5E1hf6Zd16aEpJ6

In [13]:
pip install import_ipynb gmic inhagcutils

  Using cached gmic-0.0.2.tar.gz (1.3 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gmic
  Running setup.py clean for gmic
Failed to build gmic
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (gmic)


In [17]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
force_setup = False

pip_packages = 'typer rich gmic'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

%cd /content/
!git clone https://github.com/olaviinha/ESRGAN.git Colab-ESRGAN
!git clone https://github.com/olaviinha/BlurDetection2.git
%cd "/content/Colab-ESRGAN"

dir_tmp = '/content/tmp/'
dir_mask = '/content/tmp/mask/'
dir_input = '/content/Colab-ESRGAN/input/'
dir_dejpeg = '/content/Colab-ESRGAN/dejpeg/'
dir_upscaled = '/content/Colab-ESRGAN/upscaled/'
dir_output = '/content/Colab-ESRGAN/output/'
dir_models = '/content/Colab-ESRGAN/models/'
for x in [dir_tmp, dir_mask, dir_input, dir_dejpeg, dir_upscaled, dir_output, dir_models,'/content/dejpeg/']:
  if not os.path.isdir(x):
    os.makedirs(x,exist_ok=True)


/content
fatal: destination path 'Colab-ESRGAN' already exists and is not an empty directory.
fatal: destination path 'BlurDetection2' already exists and is not an empty directory.
/content/Colab-ESRGAN


In [28]:
# ✅ SETUP: Install necessary system tools
!apt-get update
!apt-get install -y gmic imagemagick

# ✅ IMPORTS AND GLOBALS
import os
import sys
from glob import glob
from PIL import Image
import cv2

# Your helper functions (op, c, fix_path, clean_dirs, etc.) should be defined somewhere before running this block
# We'll assume they're available in your Colab notebook.

# ✅ PARAMETERS (replace with your own or use widgets)
image_or_directory_to_upscale = "pics_frans/image.jpeg"
output_dir = ""
model = "ESRGAN_and_FatalPixels"
interpolation = 53
sharpen = True
dejpeg = True
models_in_filename = False
local_models_dir = ""

# ✅ FILE PATHS
drive_root = "/content/drive/MyDrive/"
dir_tmp = "/content/tmp/"
dir_input = "/content/input/"
dir_mask = "/content/mask/"
dir_dejpeg = "/content/dejpeg/"
dir_upscaled = "/content/upscaled/"
dir_output = "/content/output/"
dir_models = "/content/Colab-ESRGAN/models/"

# ✅ INITIAL SETUP
downloaded_models = []
def dlfail():
  print("Download failed. Exiting.")
  sys.exit()

# ✅ INSTALL MODELS (skip if already downloaded)
os.makedirs(dir_models, exist_ok=True)

# Example for ESRGAN
if 'ESRGAN' in model and not os.path.exists(f"{dir_models}/RRDB_ESRGAN_x4.pth"):
  !wget -O {dir_models}/RRDB_ESRGAN_x4.pth https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-
  downloaded_models.append('ESRGAN')

# ✅ GET INPUT IMAGES
image_or_directory_to_upscale = os.path.join(drive_root, image_or_directory_to_upscale)
if os.path.isfile(image_or_directory_to_upscale):
  input_images = [image_or_directory_to_upscale]
elif os.path.isdir(image_or_directory_to_upscale):
  input_images = sorted(glob(image_or_directory_to_upscale + '/*.jpg'))
else:
  print("Input path not found")
  sys.exit()

if not output_dir:
  output_dir = os.path.dirname(input_images[0])
os.makedirs(output_dir, exist_ok=True)

# ✅ PROCESS IMAGES
for input_image in input_images:
  print(f"Processing: {input_image}")
  os.makedirs(dir_input, exist_ok=True)
  os.makedirs(dir_output, exist_ok=True)

  state = input_image

  if dejpeg:
    print("Running DeJPEG...")
    !cp "{state}" "{dir_input}"
    !python upscale.py 1x_DeJpeg_Fatality_PlusULTRA.pth --input "{dir_input}" --output "{dir_dejpeg}"
    state = os.path.join(dir_dejpeg, os.listdir(dir_dejpeg)[0])

  if sharpen:
    print("Sharpening...")
    tmp_file = os.path.join(dir_tmp, os.path.basename(input_image) + '_sharpened.png')
    !gmic "{state}" -fx_mighty_details 35,1,2,1,11,0 -output "{tmp_file}"
    state = tmp_file

  print("Upscaling with model...")
  model_file = "RRDB_ESRGAN_x4.pth"  # Simplified for this example
  !cp "{state}" "{dir_input}"
  !python upscale.py "{model_file}" --input "{dir_input}" --output "{dir_upscaled}"
  final_image = os.path.join(dir_upscaled, os.listdir(dir_upscaled)[0])

  # Save final image to output dir
  final_file = os.path.join(output_dir, os.path.basename(input_image).replace('.jpg', '_upscaled.png'))
  !cp "{final_image}" "{final_file}"

  print(f"Saved: {final_file}\n")

print("\nFIN.")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

IndexError: list index out of range